In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def ph_to_h2so4_concentration(pH, temperature=25):
    """
    Convert pH to H2SO4 concentration (rough approximation)
    
    Parameters:
    pH: measured pH of solution
    temperature: temperature in Celsius (default 25)
    
    Returns:
    H2SO4 concentration in mol/L
    
    Note: This is a simplified calculation. For accurate results,
    you'd need to account for H2SO4's two dissociation constants
    and activity coefficients.
    """
    # H+ concentration from pH
    h_conc = 10**(-pH)
    
    # Rough approximation: assume H2SO4 dissociates completely
    # H2SO4 -> H+ + HSO4- (first dissociation is strong)
    # HSO4- <-> H+ + SO4^2- (second dissociation is weaker)
    
    # For simplicity, assume most H+ comes from first dissociation
    h2so4_conc = h_conc  # This is a rough approximation
    
    return h2so4_conc

In [ ]:
def cuso4_conductivity_from_calibration(concentration):
    """
    Convert CuSO4 concentration to conductivity using a calibration curve
    
    This is a placeholder function - you'd need actual calibration data
    
    Parameters:
    concentration: CuSO4 concentration in mol/L
    
    Returns:
    Conductivity contribution in mS/cm (or your preferred units)
    """
    # Placeholder linear relationship - replace with your actual calibration data
    # Typical values might be around 100-200 mS/cm per mol/L for CuSO4
    conductivity_per_molar = 150  # mS/cm per mol/L (example value)
    
    return concentration * conductivity_per_molar

In [ ]:
def h2so4_conductivity_from_calibration(concentration):
    """
    Convert H2SO4 concentration to conductivity using a calibration curve
    
    This is a placeholder function - you'd need actual calibration data
    
    Parameters:
    concentration: H2SO4 concentration in mol/L
    
    Returns:
    Conductivity contribution in mS/cm
    """
    # Placeholder relationship - H2SO4 is highly conductive
    # Typical values might be around 400-800 mS/cm per mol/L for H2SO4
    conductivity_per_molar = 600  # mS/cm per mol/L (example value)
    
    return concentration * conductivity_per_molar

In [ ]:
def calculate_cuso4_concentration_method1(total_conductivity, pH, correction_factor=0.9, temperature=25):
    """
    Calculate CuSO4 concentration using conductivity Method 1
    
    Parameters:
    total_conductivity: measured total conductivity in mS/cm
    pH: measured pH of the solution
    correction_factor: accounts for non-ideal mixing effects (default 0.9)
    temperature: temperature in Celsius
    
    Returns:
    dict with calculated concentrations and intermediate values
    """
    
    # Step 1: Calculate H2SO4 concentration from pH
    h2so4_conc = ph_to_h2so4_concentration(pH, temperature)
    
    # Step 2: Calculate expected H2SO4 conductivity contribution
    h2so4_conductivity = h2so4_conductivity_from_calibration(h2so4_conc)
    
    # Step 3: Calculate CuSO4 conductivity contribution
    # Account for non-ideal mixing with correction factor
    cuso4_conductivity = (total_conductivity - h2so4_conductivity)
    
    # Step 4: Convert CuSO4 conductivity to concentration
    # This requires inverting your calibration curve
    # For linear relationship: conductivity = slope * concentration
    conductivity_per_molar_cuso4 = 150  # from calibration function above
    cuso4_conc = cuso4_conductivity / conductivity_per_molar_cuso4
    
    # Convert to different units
    cuso4_mass_conc = cuso4_conc * 159.61  # g/L anhydrous CuSO4
    cuso4_hydrated_mass_conc = cuso4_conc * 249.68  # g/L CuSO4·5H2O
    
    results = {
        'pH': pH,
        'H2SO4_concentration_mol_L': h2so4_conc,
        'H2SO4_conductivity_mS_cm': h2so4_conductivity,
        'CuSO4_conductivity_mS_cm': cuso4_conductivity,
        'CuSO4_concentration_mol_L': cuso4_conc,
        'CuSO4_concentration_g_L': cuso4_mass_conc,
        'CuSO4_hydrated_concentration_g_L': cuso4_hydrated_mass_conc,
        'total_conductivity_measured': total_conductivity,
        'correction_factor_used': correction_factor
    }
    
    return results

In [ ]:
def plot_calibration_curves():
    """
    Plot example calibration curves for CuSO4 and H2SO4
    """
    concentrations = np.linspace(0, 1, 100)
    
    cuso4_conductivities = [cuso4_conductivity_from_calibration(c) for c in concentrations]
    h2so4_conductivities = [h2so4_conductivity_from_calibration(c) for c in concentrations]
    
    plt.figure(figsize=(10, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(concentrations, cuso4_conductivities, 'b-', linewidth=2, label='CuSO4')
    plt.xlabel('Concentration (mol/L)')
    plt.ylabel('Conductivity (mS/cm)')
    plt.title('CuSO4 Calibration Curve')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(concentrations, h2so4_conductivities, 'r-', linewidth=2, label='H2SO4')
    plt.xlabel('Concentration (mol/L)')
    plt.ylabel('Conductivity (mS/cm)')
    plt.title('H2SO4 Calibration Curve')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Example measurements
measured_conductivity = 250  # mS/cm
measured_pH = 1.5

# Calculate CuSO4 concentration
results = calculate_cuso4_concentration_method1(
    total_conductivity=measured_conductivity,
    pH=measured_pH,
    correction_factor=0.9  # adjust based on your system
)

# Print results
print("Conductivity Method 1 Results:")
print("=" * 40)
print(f"Measured conductivity: {results['total_conductivity_measured']:.1f} mS/cm")
print(f"Measured pH: {results['pH']:.2f}")
print(f"Correction factor: {results['correction_factor_used']:.2f}")
print()
print("Calculated H2SO4:")
print(f"  Concentration: {results['H2SO4_concentration_mol_L']:.4f} mol/L")
print(f"  Conductivity contribution: {results['H2SO4_conductivity_mS_cm']:.1f} mS/cm")
print()
print("Calculated CuSO4:")
print(f"  Concentration: {results['CuSO4_concentration_mol_L']:.4f} mol/L")
print(f"  Concentration: {results['CuSO4_concentration_g_L']:.2f} g/L (anhydrous)")
print(f"  Concentration: {results['CuSO4_hydrated_concentration_g_L']:.2f} g/L (pentahydrate)")
print(f"  Conductivity contribution: {results['CuSO4_conductivity_mS_cm']:.1f} mS/cm")

# Plot example calibration curves
plot_calibration_curves()

print("\nIMPORTANT NOTES:")
print("1. Replace the placeholder calibration functions with your actual data")
print("2. The pH to H2SO4 conversion is simplified - use titration for accuracy")
print("3. Adjust the correction factor based on experimental validation")
print("4. This method works best when H2SO4 concentration is known accurately")